In [ ]:
from __future__ import print_function, division
from collections import defaultdict, OrderedDict
import gzip
import pickle
import json
import re

import uproot
%matplotlib nbagg
import matplotlib.pyplot as plt
import numpy as np

from fnal_column_analysis_tools import hist
from fnal_column_analysis_tools.hist import plot

In [ ]:
import scipy.interpolate

def roc(hist, bkg, sig, direction=-1):
    ax = hist.axes()[-1]
    bkgvals = hist.project("process", bkg).values()[()]
    sigvals = hist.project("process", sig).values()[()]
    bkgeff_cut = np.cumsum(bkgvals[::direction])
    bkgeff_cut = bkgeff_cut[::direction] / bkgeff_cut[-1]
    sigeff_cut = np.cumsum(sigvals[::direction])
    sigeff_cut = sigeff_cut[::direction] / sigeff_cut[-1]
    interp = scipy.interpolate.interp1d(ax.centers(), np.c_[bkgeff_cut, sigeff_cut], axis=0, )
    return ax.centers(), interp

In [ ]:
with gzip.open("hists.pkl.gz") as fin:
    hists_unmapped = pickle.load(fin)

In [ ]:
h=hists_unmapped['hsculpt_SR'].project("AK8Puppijet0_pt", 450., overflow='over') \
              .sum("AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb", overflow='all') \
              .project("AK8Puppijet0_deepdoubleb", slice(0.89, None), overflow='over') \
              .project("AK8Puppijet0_msd", slice(96,138))
print("%80s"%"dataset", "V(lv)", " No V", "V(qq)", "W(ud)b", "W(cs)b")
for k,v in h.values(overflow='all').items():
    v *= 41.1
    print("%80s %5.0f %5.0f %5.0f %5.0f %5.0f" % (k[0], v[0], v[1], v[2:5].sum(), v[5], v[6]))

In [ ]:
h2 = hists_unmapped["hjetpt"].sum("dataset", "AK8Puppijet0_isHadronicV")
w = h2.identifiers("AK8Puppijet0_pt", overflow='allnan')
h2[w[-2]].identifiers("AK8Puppijet0_pt", overflow='allnan')

In [ ]:
h3 = hists_unmapped["hjetpt"].sum("dataset", "AK8Puppijet0_pt")
w3 = h3.identifiers("AK8Puppijet0_isHadronicV", overflow='allnan')
w3
#h3[w3[-2]].identifiers("AK8Puppijet0_isHadronicV", overflow='allnan')

In [ ]:
process = hist.Cat("process", "Process", sorting='placement')
process_cats = ("dataset", "AK8Puppijet0_isHadronicV")
process_map = OrderedDict()
#process_map["QCDinZ"] = ("ZJets*", 0)
#process_map["QCDinW"] = ("WJets*", 0)

process_map["Hcc"] = ("GluGluHToCC", 2)
process_map["Hbb"] = ("GluGluHToBB", 3)
process_map["Wqq"] = ("WJetsToQQ_HT*", 1)
process_map["Wcs"] = ("WJetsToQQ_HT*", 2)
process_map["Zqq"] = ("ZJetsToQQ_HT*", 1)
process_map["Zcc"] = ("ZJetsToQQ_HT*", 2)
process_map["Zbb"] = ("ZJetsToQQ_HT*", 3)
top = re.compile("(?:ST_|TTTo)")
process_map["W top"] = (top, slice(1,9))
process_map["Merged top"] = (top, slice(9,11))
process_map["QCD"] = ("QCD*", slice(None))
unmatched = re.compile("(?:[WZ]JetsToQQ|ST_|TTTo|GluGluHTo)")
process_map["Unmatched"] = (unmatched, 0)


hists = {}
for key in hists_unmapped.keys():
    hists[key] = hists_unmapped[key].group(process, process_cats, process_map)

In [ ]:
fill_opts = {'edgecolor': (0,0,0,0.3), 'alpha': 0.8}
fillerr = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'none', 'edgecolor':(0,0,0,.5), 'linewidth': 0}

fig, ax, _ = plot.plot1d(hists['hjetpt_SR'], overlay="process", stack=True, fill_opts=fill_opts, error_opts=fillerr)
ax.set_yscale('log')
ax.set_ylim(.1,None)

In [ ]:
def describe(hist):
    for ax in hist.axes():
        print("Axis:", ax)
        print("    Bins:", hist.identifiers(ax))

proj = lambda h: h.project("AK8Puppijet0_pt", 450., overflow='over').project("AK8Puppijet0_msd", slice(100,140))

#h = proj(hists['opposite_ak4_leadingDeepCSV_SR'])
h = proj(hists['opposite_ak8_msd_SR'])
h.axis("opposite_ak8_msd").label = r"Jet $m_{sd}$"
noqcd = re.compile("(?!QCD$|Hcc).*")
fig2, ax2, _ = plot.plot1d(h["Unm*"], overlay="process", overflow='over', stack=True, fill_opts=fill_opts, error_opts=fillerr)


In [ ]:
import matplotlib.patheffects
fig, ax = plt.subplots(1,1)
sig = 'Hbb'
#bkg = 'Merged top'
#bkgname = 'Merged top'
bkg = re.compile(".* top")
bkgname = 'Top background'

btagvals, btagroc = roc(proj(hists['opposite_ak4_leadingDeepCSV_SR']), bkg, sig, direction=1)
bc = ax.plot(*btagroc(btagvals).T, label='Max. AK4 DeepCSV')
deepcsv_wps = [0.1522, 0.4941, 0.8001]
broc_wp = btagroc(deepcsv_wps)
ax.plot(*broc_wp.T, marker='*', linestyle='none', color=bc[0].get_color(), markersize=10)
for i, name in enumerate('LMT'):
    txt = ax.text(*broc_wp[i], name, verticalalignment='top', fontweight='bold')
    txt.set_path_effects([matplotlib.patheffects.withStroke(linewidth=1.5, foreground='w')])

n3vals, n3roc = roc(proj(hists['opposite_ak8_n3sdb1_SR']), bkg, sig, direction=-1)
ax.plot(*n3roc(n3vals).T, label='Subleading AK8 $N_{3}$')

tau32vals, tau32roc = roc(proj(hists['opposite_ak8_tau32_SR']), bkg, sig, direction=-1)
ax.plot(*tau32roc(tau32vals).T, label=r'Subleading AK8 $\tau_{32}$')

msdvals, msdroc = roc(proj(hists['opposite_ak8_msd_SR']), bkg, sig, direction=1)
ax.plot(*msdroc(msdvals).T, label=r'Subleading AK8 $m_{sd}$')

njetvals, njetroc = roc(proj(hists['njets_ak4_SR']), bkg, sig, direction=1)
ax.plot(*njetroc(njetvals).T, label=r'Number AK4 jets')

metvals, metroc = roc(proj(hists['pfmet_nminus1_SR']), bkg, sig, direction=1)
mc = ax.plot(*metroc(metvals).T, label=r'PF $p_{T}^{miss}$')
ax.plot(*metroc(140), marker='*', linestyle='none', color=mc[0].get_color(), markersize=10)
txt = ax.text(*metroc(140), '140', verticalalignment='top', fontweight='bold')
txt.set_path_effects([matplotlib.patheffects.withStroke(linewidth=1.5, foreground='w')])

ax.plot([0,1], [0,1], linestyle='--', color='grey')

ax.legend(title="Opposite-hemisphere cut")
ax.autoscale(tight=True)
ax.set_ylabel(sig+' efficiency')
ax.set_xlabel(bkgname+' efficiency')

In [ ]:
nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in hists.values())
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in hists.values())
print("Now %.1fM bins" % (nbins/1e6, ))
print("Nonzero bins: %.1f%%" % (100*nfilled/nbins, ))

In [ ]:
if 'htagtensor_SR' in hists:
    with gzip.open("tagtensor.pkl.gz", "wb") as fout:
        pickle.dump(hists['htagtensor_SR'], fout, protocol=2)  # for python2 compatibility

In [ ]:
# check if unmatched jet in V sample is any different than QCD
# answer was no
if False:
    hqcd_comp = hists['hsculpt_b'].sum("AK8Puppijet0_msd")["QCD*"]
    fig,axes = plot.plotgrid(hqcd_comp, col="AK8Puppijet0_pt", overlay="process", error_opts={}, density=True)
    axes[0][0].set_ylim(1e-3,None)
    axes[0][0].set_yscale('log')

In [ ]:
x = hists['htagtensor'].sum("AK8Puppijet0_pt", "AK8Puppijet0_msd").sum("AK8Puppijet0_N2sdb1_ddt", overflow='under')
pvals = x[:,:,0.6:,0.:].sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb").values()
avals = x.sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb").values()
evals = {k:pvals[k]/avals[k] for k in pvals.keys()}
evals

In [ ]:
fig, ax, _ = plot.plot1d(x.sum("AK8Puppijet0_deepdoubleb").project("AK8Puppijet0_deepdoublecvb", slice(0.15,None)),
            overlay="process", error_opts={}, density=True
           )

In [ ]:
plot_opts = {'error_opts': {}, 'overflow': 'none', 'overlay_overflow': 'all', 'density': True}

print("msd bin:", hists['htagtensor'].axis("AK8Puppijet0_msd")[1])
htagtensor = hists['htagtensor'].project("AK8Puppijet0_msd", overflow='none').project("AK8Puppijet0_N2sdb1_ddt", overflow='under')
htagtensor.label = "Density"

hb = htagtensor.sum("AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb")
fig1,_ = plot.plotgrid(hb, col="process", overlay="AK8Puppijet0_pt", **plot_opts)
fig1.savefig("plots/doubleb.pdf")

hc = htagtensor.sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublecvb")
fig2,_ = plot.plotgrid(hc, col="process", overlay="AK8Puppijet0_pt", **plot_opts)
fig2.savefig("plots/doublec.pdf")

hcvb = htagtensor.sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec")
fig3,_ = plot.plotgrid(hcvb, col="process", overlay="AK8Puppijet0_pt", **plot_opts)
fig3.savefig("plots/doublecvb.pdf")

In [ ]:
rocvals = htagtensor.project("AK8Puppijet0_pt", slice(450,None), overflow='over') \
                    .sum("AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb") \
                    .values()
qcdeff_b, zbbeff_b = roc(rocvals, 'QCD', 'Zbb')

fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.plot(zbbeff_b, qcdeff_b)
ax.set_xlabel("Z(bb) jet $\epsilon$")
ax.set_xlim(0,1)
ax.set_ylabel("QCD jet $\epsilon$")
ax.set_yscale('log')
ax.set_ylim(1e-3, 1)
ax.grid(which='both')
auc = 1+np.trapz(y=qcdeff_b, x=zbbeff_b)  # range of integration is 1 to 0
ax.set_title("$300 \leq p_{T,j} < 2000, 40 \leq m_{sd} < 160$, AUC: %.3f" % auc)
fig.savefig("plots/Zbb_roc.pdf")

In [ ]:
rocvals = htagtensor.project("AK8Puppijet0_pt", slice(450,None), overflow='over') \
                    .sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublecvb") \
                    .values()
qcdeff_c, zcceff_c = roc(rocvals, 'QCD', 'Hcc')

fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.plot(zcceff_c, qcdeff_c)
ax.set_xlabel("H(cc) jet $\epsilon$")
ax.set_xlim(0,1)
ax.set_ylabel("QCD jet $\epsilon$")
ax.set_yscale('log')
ax.set_ylim(1e-3, 1)
ax.grid(which='both')
auc = 1+np.trapz(y=qcdeff_c, x=zcceff_c)
ax.set_title("$450 \leq p_{T,j}, 40 \leq m_{sd} < 160$, AUC: %.3f" % auc)
fig.savefig("plots/Hcc_roc.pdf")

In [ ]:
rocvals = htagtensor.project("AK8Puppijet0_pt", slice(450,None), overflow='over') \
                    .sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec") \
                    .values()
zbbeff_cvb, zcceff_cvb = roc(rocvals, 'Zbb', 'Zcc')

fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.plot(zcceff_cvb, zbbeff_cvb)
ax.set_xlabel("Z(cc) jet $\epsilon$")
ax.set_xlim(0,1)
ax.set_ylabel("Z(bb) jet $\epsilon$")
ax.set_yscale('log')
ax.set_ylim(1e-3, 1)
ax.grid(which='both')
auc = 1+np.trapz(y=zbbeff_cvb, x=zcceff_cvb)
ax.set_title("$300 \leq p_{T,j} < 2000, 40 \leq m_{sd} < 160$, AUC: %.3f" % auc)
fig.savefig("plots/Zcvb_roc.pdf")

In [ ]:
def rebin_quantile(ax, vals, quantiles, cmpLess=False):
    quantiles.sort()
    idir = 1 if cmpLess else -1
    cdf = vals[::idir].cumsum()
    cdf /= cdf[-1]
    bins = np.searchsorted(cdf, quantiles)
    if bins[0] > 0:
        bins = np.insert(bins, 0, 0)
    if bins[-1] < len(ax.edges()):
        bins = np.append(bins, -1)
    edges = ax.edges()[bins]
    qbin = hist.Bin(ax.name, ax.label, edges)
    return qbin


In [ ]:
n2name = "AK8Puppijet0_N2sdb1_ddt"
rocn2 = hists['hsculpt'].project("AK8Puppijet0_pt", slice(450., None)) \
            .project("AK8Puppijet0_msd")
n2 = rocn2.axis(n2name)

n2proj = rocn2.sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb", overflow='all')

quantiles = np.array([0.01, 0.1, 0.2, 0.5])
n2_coarse = rebin_quantile(n2, n2proj.values()[('QCD',)], quantiles, True)
print(n2_coarse.edges())


figq, axq = plt.subplots(1,1, figsize=(5,5))
plot.plot1d(axq, n2proj['QCD'], n2, error_opts={}, density=True)
for i,e in enumerate(n2_coarse.edges()[1:-1]):
    axq.axvline(e, linestyle='--', color='k')
    axq.text(e, axq.get_ylim()[1], "QCD $\epsilon=%.2f$" % quantiles[i], rotation=90, va='top', ha='right')
axq.legend()
axq.set_xlim(-.25,.25)
axq.set_ylim(0,None)
axq.set_title("$450 \leq p_{T,j}, 40 \leq m_{sd} < 160$")
#figq.savefig("plots/n2_qcdquantiles.pdf")

In [ ]:
rocvals = rocn2.rebin(n2, n2_coarse).values()
for k in rocvals.keys(): rocvals[k] = rocvals[k].cumsum(axis=-1)

fig, ax = plt.subplots(1,1, figsize=(5,5))
for i in range(n2_coarse.size-3):
    n2bin = n2_coarse[i+1]
    qcdeff_b, zbbeff_b = roc({k:v[:,i] for k,v in rocvals.items()}, 'QCD', 'Zbb')
    ax.plot(zbbeff_b, qcdeff_b, label="%.12g" % n2bin.hi)

ax.set_xlabel("Z(bb) jet $\epsilon$")
ax.set_xlim(0,1)
ax.set_ylabel("QCD jet $\epsilon$")
ax.set_yscale('log')
ax.set_ylim(1e-3, 1)
#ax.set_ylim(0,1)
ax.legend(title="N2DDT cut")
ax.grid(which='both')
ax.set_title("$450 \leq p_{T,j}, 40 \leq m_{sd} < 160$")

fig.savefig("plots/doubleb_roc_n2.pdf")

In [ ]:
hists['hsculpt_SR'].identifiers("AK8Puppijet0_deepdoublec")

In [ ]:
ptbins = hists['hsculpt_SR'].axis("AK8Puppijet0_pt").identifiers()
ptbins

In [ ]:
i = 0
sculpt = hists['hsculpt_SR'] \
              .project("AK8Puppijet0_pt", ptbins[i], overflow='over') \
              .sum("AK8Puppijet0_deepdoubleb", overflow='all') \
              .project("AK8Puppijet0_deepdoublec", slice(0.87, None), overflow='over') \
              .project("AK8Puppijet0_deepdoublecvb", slice(0.20, None), overflow='over')
#               .sum("AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb", overflow='all') \
#               .project("AK8Puppijet0_deepdoubleb", slice(0.89, None), overflow='over')

lumi = 41.1
sculpt.scale(lumi)
sculpt.label = "Events / 7 GeV"
fillerr = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'none', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
noqcd = re.compile("(?!QCD$).*")
#fig, ax, _ = plot.plot1d(sculpt[noqcd,:], overlay="process", stack=True, fill_opts=fill_opts, error_opts=fillerr)
subset = re.compile("H.*")
sculpt.label = 'Density'
fig, ax, _ = plot.plot1d(sculpt[subset], overlay="process", error_opts={}, density=False)
ax.autoscale(axis='x', tight=True)
#ax.set_xlim(40,166)
ax.set_ylim(0, None)
# ax.legend(title='DDC SR, $p_{T} \in $%s'%str(ptbins[i]))
ax.legend(title='DDC SR, $p_{T} > 450$')
lumi = plt.text(1., 1., r"%.1f fb$^{-1}$ (13 TeV)" % lumi, fontsize=14, horizontalalignment='right', verticalalignment='bottom', transform=ax.transAxes)

#fig.savefig('cc_sr_density.pdf')

In [ ]:
h = hists['hsculpt_SR'].project("process", "Merged top") \
              .project("AK8Puppijet0_pt", slice(None)) \
              .sum("AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb", overflow='all')
h.label = 'Density'
fig,ax,_=plot.plot2d(h, xaxis="AK8Puppijet0_deepdoubleb", patch_opts={}, density=True)
ax.set_title("Merged top")

In [ ]:
hwps = hists['hsculpt'] \
  .project(n2, slice(None, 0.), overflow='under') \
  .project("AK8Puppijet0_pt", slice(450., None)) \
  .project("AK8Puppijet0_msd", slice(75, 110)) #slice(103, 145))

In [ ]:
vals = hwps.values(overflow='all')
axes = hwps.axes()[1:]
sig = 40.*sum([v for k,v in vals.items() if 'Hbb' in k[0]])
bkg = 40.*sum([v for k,v in vals.items() if 'Hbb' not in k[0]])

In [ ]:
v = []
for icvl, cvl in enumerate(axes[1].identifiers(overflow='all')):
    for icvb, cvb in enumerate(axes[2].identifiers(overflow='all')):
        ssum = sig[:,icvl:,icvb:].sum()
        bsum = bkg[:,icvl:,icvb:].sum()
        v.append((ssum/(np.sqrt(bsum)+1), cvl.lo, cvb.lo))

In [ ]:
v = [x for x in v if x[1]!=0.84 and x[2]!=0.12]

In [ ]:
v.sort(key=lambda x: x[0], reverse=True)
for i in v[:10]:
    print("CvL >= {1} CvB >= {2:.2f} S/(sqrtB+1) = {0:.4f}".format(*i))

In [ ]:
hwps = hists['hsculpt_SR'] \
  .project("AK8Puppijet0_pt", slice(450., None)) \
  .project("AK8Puppijet0_msd", slice(103, 145))
#   .project(n2, slice(None, 0.), overflow='under') \

vals = hwps.values(overflow='all')
axes = hwps.axes()[1:]
sig = 40.*sum([v for k,v in vals.items() if 'Hbb' in k[0]])
bkg = 40.*sum([v for k,v in vals.items() if 'Hbb' not in k[0]])

v = []
for ibvl, bvl in enumerate(axes[0].identifiers(overflow='all')):
    ssum = sig[ibvl:,:,:].sum()
    bsum = bkg[ibvl:,:,:].sum()
    v.append((ssum/(np.sqrt(bsum)+1), bvl.lo))

v.sort(key=lambda x: x[0], reverse=True)
for i in v[:10]:
    print("BvL >= {1} S/(sqrtB+1) = {0:.4f}".format(*i))

In [ ]:
sculpt = hists['hsculpt_SR']
hsculpt = sculpt.sum("AK8Puppijet0_deepdoublec","AK8Puppijet0_deepdoublecvb", overflow='all') \
    .project("AK8Puppijet0_pt", slice(450, None), overflow='over') \
    .project("process", "QCD")

for ddb_bin in hsculpt.identifiers("AK8Puppijet0_deepdoubleb"):
    fig, ax = plt.subplots(1,1)
    wp = ddb_bin.lo
    hfail = hsculpt.project("AK8Puppijet0_deepdoubleb", slice(None, wp), overflow='under')
    hpass = hsculpt.project("AK8Puppijet0_deepdoubleb", slice(wp, None), overflow='over')
    plot.plot1d(hfail, ax=ax, clear=False, error_opts={'label': 'QCD fail'}, density=True)
    plot.plot1d(hpass, ax=ax, clear=False, error_opts={'label': 'QCD pass'}, density=True)
    ax.legend(title="ddb WP=%.2f" % wp)
    ax.set_title("AK8 $p_{T}\geq 450$ GeV, $N_{2}^{ddt}<0$")
    fig.savefig("scuplt_ddb_wp%s.pdf" % str(wp).replace('.','p'))

In [ ]:
fig, ax = plt.subplots(1,1)
plot.plot2d(hsculpt, xaxis="AK8Puppijet0_msd", ax=ax, patch_opts={}, density=True)

In [ ]:
hn2 = hists['hsculpt']["QCD"] \
    .sum("process", "AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb") \
    .project("AK8Puppijet0_pt", slice(450,None))

plot.plot2d(hn2, xaxis="AK8Puppijet0_msd", xoverflow='over', patch_opts={})

In [ ]:
# this is just a histogram I already had
hn2 = hists['hsculpt'] \
    .sum("AK8Puppijet0_deepdoubleb", "AK8Puppijet0_deepdoublec", "AK8Puppijet0_deepdoublecvb") \
    .project("AK8Puppijet0_pt", slice(450,None))
print(hn2)

# we'll optimize s/sqrt(b) as a function of the mass and n2 cut
cumsum_directions = [-1, 1] # msd >= x, n2 < x

hsignif = hn2.project("process")
hsignif.clear() # just to get the axis definitions
hsignif.label = '$S/\sqrt{B}$'

arrays = hn2.values(overflow='allnan')
ordering = tuple(slice(None,None,d) for d in cumsum_directions)
qcd_cdf = arrays[('QCD',)][ordering].cumsum(axis=0).cumsum(axis=1)[ordering]
hbb_cdf = arrays[('Hbb',)][ordering].cumsum(axis=0).cumsum(axis=1)[ordering]
signif = hbb_cdf / np.maximum(1, np.sqrt(qcd_cdf))
hsignif._sumw = {():signif}
plot.plot2d(hsignif, xaxis="AK8Puppijet0_msd", xoverflow='over', patch_opts={})

In [ ]:
plot.plot2d(hn2.project('process', 'QCD'), xaxis="AK8Puppijet0_msd", patch_opts={})
plot.plot2d(hn2.project('process', 'Hbb'), xaxis="AK8Puppijet0_msd", patch_opts={})